In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils.dataframe import column as column_utils


from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report

import lightgbm as lgb

np.set_printoptions(threshold=np.nan)



plt.rcParams["figure.figsize"] = (24,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import missingno as msno
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import datetime
from scipy import signal
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn import linear_model



InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import warnings
warnings.filterwarnings('ignore')
import dask.dataframe as dd

In [2]:
df_train = dd.read_hdf("data/train_1.h5","table")
df_train = dd.from_pandas(df_train.compute(),npartitions=64)
gc.collect()


217

In [3]:
GROUP_BY_NEXT_CLICKS_v0 = [
    
    # V1
    {'groupby': ['ip']}
]

GROUP_BY_NEXT_CLICKS_v1 = [
    
    # V1
    {'groupby': ['ip']},
    {'groupby': ['app']},
    {'groupby': ['device']},
    {'groupby': ['ip', 'app']},
    {'groupby': ['ip', 'channel']},
    {'groupby': ['ip', 'os']},
    # V3
    {'groupby': ['ip', 'app', 'device', 'os', 'channel']},
    {'groupby': ['ip', 'os', 'device']},
    {'groupby': ['ip', 'os', 'device', 'app']}
]

GROUP_BY_NEXT_CLICKS_v2 = [
    
    {'groupby': ['ip','hour']},
    {'groupby': ['ip', 'app', 'device', 'os', 'channel','hour']},
    {'groupby': ['ip', 'os', 'device','hour']},
    {'groupby': ['ip', 'os', 'device', 'app','hour']},
    {'groupby': ['ip', 'app','hour']},
    {'groupby': ['ip', 'channel','hour']},
    {'groupby': ['ip', 'os','hour']},
    {'groupby': ['app', 'device', 'os', 'channel']},
    {'groupby': ['app', 'device', 'os']},
    {'groupby': ['app', 'device', 'os','hour']}
]

In [4]:
# dask test example
pdf = pd.DataFrame({"A":[1,1,1,2,2,2],"B":[1,5,7,12,19,20]})
ddf = dd.from_pandas(pdf,npartitions=1)

def differ(x):
    return x.diff()

def grad(x):
    if(x.size<=1):
        return pd.Series(0,index=x.index)
    return pd.Series(np.gradient(x,axis=0),index=x.index)
gp = ddf.groupby(["A"]).B
a=gp.apply(differ).compute()
b=gp.apply(grad).compute()



ddf['la'] = a
ddf['lb'] = b

# ddf.assign(la=a).compute()
ddf.head(10)

,A,B,la,lb
0,1,1,NaN,4.0
1,1,5,4.0,3.0
2,1,7,2.0,2.0
3,2,12,NaN,7.0
4,2,19,7.0,4.0
5,2,20,1.0,1.0


In [6]:


def add_time_to_next_click(GROUP_BY_NEXT_CLICKS,df):
    gc.collect()
    for spec in GROUP_BY_NEXT_CLICKS:
        next_click = 'nc_{}'.format('_'.join(spec['groupby']))
        grad_click = 'gc_{}'.format('_'.join(spec['groupby']))
        all_features = spec['groupby'] + ['click_time']

        print(f"Grouping by {spec['groupby']}, and saving time to next click in: {next_click}")
        # group = df[all_features].groupby(spec['groupby']).click_time  
        # gp = group.transform(lambda x: x.diff().shift(-1))
        def differ(x):
            return x.diff().shift(-1)
        
        def grad(x):
            if(x.size<=1):
                return pd.Series(np.nan,index=x.index)
            return pd.Series(np.gradient(x,axis=0),index=x.index)
        group = df.groupby(spec['groupby']).click_time
        gp1 = group.apply(differ).compute()
        gp2 = group.apply(grad).compute()
        
        df[next_click] = gp1
        df[grad_click] = gp2
        
        gp=None
        del gp
        group = None
        del group
        gc.collect()
    


In [ ]:
add_time_to_next_click(GROUP_BY_NEXT_CLICKS_v1,df_train)
gc.collect()

Grouping by ['ip'], and saving time to next click in: nc_ip


In [ ]:
add_time_to_next_click(GROUP_BY_NEXT_CLICKS_v2,df_train)
gc.collect()

In [ ]:
test_df = dd.from_pandas(df_train[df_train.ip==83230].compute(),npartitions=4)
test_df.head(20)

In [ ]:
gc.collect()
df_train.to_hdf('data/train_2.h5',"table",mode='w')
gc.collect()
print("Train_2.h5 done")
df_utils.count_nulls(df_train)

In [ ]:
df_train.head()
df_train = None
gc.collect()
del df_train
test_df = None
del test_df
gc.collect()

In [ ]:
df_test = pd.read_hdf("data/test_1.h5","table")
df_test = dd.from_pandas(df_test.compute(),npartitions=64)
gc.collect()

In [ ]:
add_time_to_next_click(GROUP_BY_NEXT_CLICKS_v1,df_test)
gc.collect()

In [ ]:
add_time_to_next_click(GROUP_BY_NEXT_CLICKS_v2,df_test)
gc.collect()

In [ ]:
gc.collect()
df_test.to_hdf('data/test_2.h5',"table",mode='w')
gc.collect()
print("Test_2.h5 done")

In [ ]:

df_test = None
gc.collect()
del df_test
gc.collect()
